# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [1]:
import os
import glob
import json
import psycopg2
import pandas as pd
from sql_queries import *
from create_tables import create_database

In [2]:
os.chdir('/home/workspace')
print(os.getcwd())

/home/workspace


In [3]:
#drop and create tables

%run create_tables.py

The songplays table has been dropped.
The users table has been dropped.
The songs table has been dropped.
The artists table has been dropped.
The time table has been dropped.
The users table has been created.
The artists table has been created.
The songs table has been created.
The songplays table has been created.
The time table has been created.


In [5]:
#test etl.py code

#%run etl.py 

In [6]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student",)
conn.autocommit = True
cur = conn.cursor()

In [7]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [8]:
directories = ['data/song_data/A/A/A', 'data/song_data/A/A/B', 'data/song_data/A/A/C', 'data/song_data/A/B/A', 'data/song_data/A/B/B', 'data/song_data/A/B/C']

In [9]:
def correct_json(datafile):
    
    """
    Open json data and return a pandas dataframe.
    
    This fucntion just reads/collects data from the json file
    and creates a pandas dataframe.
    
    Parameters:
    -------------------------------------------------------
    datafile = filepath
    
    Returns:
    -------------------------------------------------------
    pandas dataframe
    
    """
    
    try:
        f = open(f'{datafile}')
        data = json.load(f)
        for i in data:
            data[i] = [data[i]]
        df_aux = pd.DataFrame(data)
        return df_aux
    except Exception as e:
        return print(e)

In [10]:
df = pd.DataFrame()  # empty datframe
for directory in directories:
    files = os.listdir(directory) # get all files
    #print()
    print(f'Amount of files: {len(files)} | Directory: {directory}')
    for file in files:
        df_aux = correct_json(f'{directory}/{file}')  # read json files
        df = df.append(df_aux)  # append to an empty dataframe
print()
df.reset_index(drop=True, inplace=True)
df.info()

Amount of files: 12 | Directory: data/song_data/A/A/A
[Errno 21] Is a directory: 'data/song_data/A/A/A/.ipynb_checkpoints'
Amount of files: 11 | Directory: data/song_data/A/A/B
Amount of files: 15 | Directory: data/song_data/A/A/C
[Errno 21] Is a directory: 'data/song_data/A/A/C/.ipynb_checkpoints'
Amount of files: 11 | Directory: data/song_data/A/B/A
Amount of files: 12 | Directory: data/song_data/A/B/B
Amount of files: 12 | Directory: data/song_data/A/B/C

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71 entries, 0 to 70
Data columns (total 10 columns):
num_songs           71 non-null int64
artist_id           71 non-null object
artist_latitude     31 non-null float64
artist_longitude    31 non-null float64
artist_location     71 non-null object
artist_name         71 non-null object
song_id             71 non-null object
title               71 non-null object
duration            71 non-null float64
year                71 non-null int64
dtypes: float64(3), int64(2), object(5)
mem

In [11]:
df.sample(5)

,num_songs,artist_id,artist_latitude,artist_longitude,artist_location,artist_name,song_id,title,duration,year
20,1,ARI3BMM1187FB4255E,38.89910,-77.02900,Washington,Alice Stuart,SOBEBDG12A58A76D60,Kassie Jones,220.78649,0
19,1,ARC43071187B990240,NaN,NaN,"Wisner, LA",Wayne Watson,SOKEJEJ12A8C13E0D0,The Urgency (LP Version),245.21098,0
45,1,AROGWRA122988FEE45,NaN,NaN,,Christos Dantis,SOSLAVG12A8C13397F,Den Pai Alo,243.82649,0
9,1,ARXR32B1187FB57099,NaN,NaN,,Gob,SOFSOCN12A8C143F5D,Face the Ashes,209.60608,2007
40,1,ARI2JSK1187FB496EF,51.50632,-0.12714,"London, England",Nick Ingman;Gavyn Wright,SODUJBS12A8C132150,Wessex Loses a Bride,111.62077,0


# #1: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [12]:
song_df = df.copy()
artists = song_df[['artist_id', 'artist_name', 'artist_location', 'artist_latitude', 'artist_longitude']].copy()  # select artist columns
print(f'Shape before: {artists.shape[0]}') 
artists.drop_duplicates(subset=['artist_id'], inplace=True)  # remove duplicates based on artist_id
print(f'Shape after: {artists.shape[0]}')
artists.sample(5)

Shape before: 71
Shape after: 69


,artist_id,artist_name,artist_location,artist_latitude,artist_longitude
19,ARC43071187B990240,Wayne Watson,"Wisner, LA",NaN,NaN
53,ARBEBBY1187B9B43DB,Tom Petty,"Gainesville, FL",NaN,NaN
47,AREDL271187FB40F44,Soul Mekanik,,NaN,NaN
45,AROGWRA122988FEE45,Christos Dantis,,NaN,NaN
30,ARGIWFO1187B9B55B7,Five Bolt Main,,NaN,NaN


In [13]:
artist_data = []
for index, row in artists.iterrows():  # iterate over dataframe to collect rows and input into list
    artist_data.append(list(row))  
artists.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 69 entries, 0 to 70
Data columns (total 5 columns):
artist_id           69 non-null object
artist_name         69 non-null object
artist_location     69 non-null object
artist_latitude     31 non-null float64
artist_longitude    31 non-null float64
dtypes: float64(2), object(3)
memory usage: 3.2+ KB


#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [14]:
inserts_artists = 0  # count inserts
for artist in artist_data: 
    cur.execute(artist_table_insert, artist)  # run query to insert data into artist table
    inserts_artists+=1
    #conn.commit()
print(f"{inserts_artists} rows inserted in artist's table")

69 rows inserted in artist's table


Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [15]:
songs = song_df[['song_id', 'title', 'artist_id', 'year', 'duration']].copy()  # select song columns
print(songs.shape[0])
songs.drop_duplicates(subset=['song_id'], inplace=True)  # remove duplicates based on song_id
print(songs.shape[0])
songs.head()

71
71


,song_id,title,artist_id,year,duration
0,SOMZWCG12A8C13C480,I Didn't Mean To,ARD7TVE1187B99BFB1,0,218.93179
1,SOUDSGM12AC9618304,Insatiable (Instrumental Version),ARNTLGG11E2835DDB9,0,266.39628
2,SOIAZJW12AB01853F1,Pink World,AR8ZCNI1187B9A069B,1984,269.81832
3,SOHKNRJ12A6701D1F8,Drop of Rain,AR10USD1187B99F3F1,0,189.57016
4,SOCIWDW12A8C13D406,Soul Deep,ARMJAGH1187FB546F3,1969,148.03546


In [16]:
song_data = []
for index, row in songs.iterrows():  # iterate over dataframe to collect rows and input into list
    song_data.append(list(row))

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [17]:
inserts_songs = 0
for list_ in song_data:
    cur.execute(song_table_insert, list_)  # run query to insert data into song table
    inserts_songs+=1
    #conn.commit()
print(f"{inserts_songs} rows inserted in song's table")

71 rows inserted in song's table


Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [18]:
file_path = ['data/log_data/2018/11']

In [19]:
df_log = pd.DataFrame()  # empty dataframe
files_log = os.listdir(file_path[0])  # get all files
for file_log in files_log:
    if file_log == '.ipynb_checkpoints':
        pass
    else:
        df_aux = pd.read_json(f'{file_path[0]}/{file_log}', lines=True) # read json data
        df_aux['userId'] = df_aux['userId'].astype(str)
        df_log = df_log.append(df_aux)  # append to an empty dataframe
print(df_log.shape)
df_log.info()

(8056, 18)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 8056 entries, 0 to 145
Data columns (total 18 columns):
artist           6820 non-null object
auth             8056 non-null object
firstName        7770 non-null object
gender           7770 non-null object
itemInSession    8056 non-null int64
lastName         7770 non-null object
length           6820 non-null float64
level            8056 non-null object
location         7770 non-null object
method           8056 non-null object
page             8056 non-null object
registration     7770 non-null float64
sessionId        8056 non-null int64
song             6820 non-null object
status           8056 non-null int64
ts               8056 non-null int64
userAgent        7770 non-null object
userId           8056 non-null object
dtypes: float64(2), int64(4), object(12)
memory usage: 1.2+ MB


In [20]:
df_log = df_log[df_log['page'] == 'NextSong']  # filter dataframe based on NextSong category 

## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [21]:
df_log['ts'] = pd.to_datetime(df_log['ts'], unit='ms')  # convert ts column to datetime data type
df_log['hour'] = df_log['ts'].dt.hour  # extract hour from ts column
df_log['day'] = df_log['ts'].dt.day  # extract day from ts column
df_log['week'] = df_log['ts'].dt.week  # extract week from ts column
df_log['month'] = df_log['ts'].dt.month  # extract month from ts column
df_log['year'] = df_log['ts'].dt.year  # extract year from ts column
df_log['weekday'] = df_log['ts'].dt.weekday  # extract weekday from ts column

df_log.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,...,status,ts,userAgent,userId,hour,day,week,month,year,weekday
0,Stephen Lynch,Logged In,Jayden,M,0,Bell,182.85669,free,"Dallas-Fort Worth-Arlington, TX",PUT,...,200,2018-11-30 00:22:07.796,Mozilla/5.0 (compatible; MSIE 10.0; Windows NT...,91,0,30,48,11,2018,4
1,Manowar,Logged In,Jacob,M,0,Klein,247.56200,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,...,200,2018-11-30 01:08:41.796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73,1,30,48,11,2018,4
2,Morcheeba,Logged In,Jacob,M,1,Klein,257.41016,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,...,200,2018-11-30 01:12:48.796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73,1,30,48,11,2018,4
3,Maroon 5,Logged In,Jacob,M,2,Klein,231.23546,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,...,200,2018-11-30 01:17:05.796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73,1,30,48,11,2018,4
4,Train,Logged In,Jacob,M,3,Klein,216.76363,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,...,200,2018-11-30 01:20:56.796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73,1,30,48,11,2018,4


In [22]:
time = df_log[df_log['page'] == 'NextSong'][['ts', 'hour', 'day', 'week', 'month', 'year', 'weekday']]  # select time columns
time.rename(columns={'ts' : 'timestamp'}, inplace=True)
time_df = time.copy()
time_df.head()

,timestamp,hour,day,week,month,year,weekday
0,2018-11-30 00:22:07.796,0,30,48,11,2018,4
1,2018-11-30 01:08:41.796,1,30,48,11,2018,4
2,2018-11-30 01:12:48.796,1,30,48,11,2018,4
3,2018-11-30 01:17:05.796,1,30,48,11,2018,4
4,2018-11-30 01:20:56.796,1,30,48,11,2018,4


In [23]:
time_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6820 entries, 0 to 145
Data columns (total 7 columns):
timestamp    6820 non-null datetime64[ns]
hour         6820 non-null int64
day          6820 non-null int64
week         6820 non-null int64
month        6820 non-null int64
year         6820 non-null int64
weekday      6820 non-null int64
dtypes: datetime64[ns](1), int64(6)
memory usage: 426.2 KB


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [24]:
inserts_time = 0
for i, row in time_df.iterrows():  
    cur.execute(time_table_insert, list(row))  # run query to insert data into time table
    inserts_time+=1
    conn.commit()
print(f"{inserts_time} rows inserted in time's table")

6820 rows inserted in time's table


Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [25]:
user_df = df_log[['userId', 'firstName', 'lastName', 'gender', 'level']].copy()  # select user columns
user_df = user_df[user_df['userId'] != '']  # remove null values
user_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6820 entries, 0 to 145
Data columns (total 5 columns):
userId       6820 non-null object
firstName    6820 non-null object
lastName     6820 non-null object
gender       6820 non-null object
level        6820 non-null object
dtypes: object(5)
memory usage: 319.7+ KB


In [26]:
user_df.head()

,userId,firstName,lastName,gender,level
0,91,Jayden,Bell,M,free
1,73,Jacob,Klein,M,paid
2,73,Jacob,Klein,M,paid
3,73,Jacob,Klein,M,paid
4,73,Jacob,Klein,M,paid


In [27]:
user_df.loc[:,'userId'] = user_df.loc[:,'userId'].astype(int)  # convert user_id column to int data type
user_df.drop_duplicates(subset=['userId'], inplace=True)  # remove duplicates based on user_id
user_df.reset_index(drop=True, inplace=True)
user_df.shape[0]

96

#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [31]:
inserts_user = 0
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, list(row))   # run query to insert data into user table
    inserts_user+=1
    conn.commit()
print(f"{inserts_user} rows inserted in user's table")    

96 rows inserted in user's table


Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [32]:
df = df_log[df_log['page'] == 'NextSong']  # filter dataframe based on NextSong category 

In [33]:
inserts_songplay = 0
for index, row in df.iterrows():

    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
    
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None

    # insert songplay record
    songplay_data = (row.ts, row.userId, row.level, songid, artistid, row.sessionId, row.location, row.userAgent)
    cur.execute(songplay_table_insert, songplay_data)
    inserts_songplay+=1
    #conn.commit()

In [34]:
print(f"{inserts_songplay} rows inserted in songplay's table")    

6820 rows inserted in songplay's table


Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [30]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.